# Autogen code executor 
Simple example of python code execution using Autogen.

Please set your OpenAI API key and AgentOps key in a local .env file for load_dotenv():
```
OPENAI_API_KEY=sk...
AGENTOPS_API_KEY=
```
or set it in the environment explicity using 
```
import os
os.environ["OPENAI_API_KEY"] = "sk..."
os.environ["AGENTOPS_API_KEY"] = ".."
```

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
!pip3 install agentops autogen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.0/352.0 kB 5.0 MB/s eta 0:00:00a 0:00:01
  Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
  Using cached docker-7.1.0-py3-none-any.whl (147 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 14.9 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import agentops

from autogen import ConversableAgent
agentops.init(os.environ.get("AGENTOPS_API_KEY"), default_tags=["autogen-example-1"])

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=8bc998e1-2500-47b2-9989-a92b88c2bdad


In [8]:
gpt4_config_list = {"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]}


In [12]:
# The code writer agent's system message is to instruct the LLM on how to
# use the Jupyter code executor with IPython kernel.
code_writer_system_message = """
You have been given coding capability to solve tasks using Python code.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
"""

import os, autogen

code_writer_agent = ConversableAgent(
    "code_writer",
    system_message=code_writer_system_message,
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    code_execution_config=False,  # Turn off code execution for this agent.
    max_consecutive_auto_reply=2,
    human_input_mode="NEVER",
)

In [13]:
code_executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the code writer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

In [15]:
import pprint

chat_result = code_executor.initiate_chat(
    code_writer_agent, message="Write Python code to calculate the 14th Fibonacci number."
)

pprint.pprint(chat_result)

Executor (to code_writer):

Write Python code to calculate the 14th Fibonacci number.

--------------------------------------------------------------------------------
code_writer (to Executor):

Here is a Python script that you can run to calculate the 14th Fibonacci number:

```python
def fibonacci(n):
    if n < 0:
        print("Incorrect input")
    elif n == 1 or n == 0:
        return n
    else:
        a, b = 0, 1
        for _ in range(2, n+1):
            a, b = b, a + b
        return b

print(fibonacci(14))
```

This script defines a function `fibonacci(n)` that calculates the nth Fibonacci number for any n greater than or equal to zero. The function uses a loop to add up the two previous Fibonacci numbers to calculate the current one.

After we define the function, we call it with `n` set to 14 to calculate the 14th Fibonacci number. The result is then printed out.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE B